In [ ]:
# Load local adapter and push to HF
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import AutoPeftModelForCausalLM


model = AutoPeftModelForCausalLM.from_pretrained("wuschelschulz/SFT_reasoning_Gemma_3_1B_unsloth", device_map="auto")
original_model = AutoModelForCausalLM.from_pretrained("unsloth/gemma-3-1b-it", device_map="auto")

tokenizer = AutoTokenizer.from_pretrained("unsloth/gemma-3-1b-it")



/root/sky_workdir/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Skipping import of cpp extensions due to incompatible torch version 2.7.0+cu126 for torchao version 0.14.1             Please see https://github.com/pytorch/ao/issues/2919 for more info
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [2]:
messages = [
    {"role": "user", "content": "What is the sum of the first 100 natural numbers?"}
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
)

In [3]:
inputs = inputs.to(model.device)

outputs = model.generate(inputs, max_new_tokens=1024)
original_outputs = original_model.generate(inputs, max_new_tokens=1024)

In [4]:
print("=" * 80)
print("BASE MODEL OUTPUT:")
print("=" * 80)
print(tokenizer.decode(original_outputs[0], skip_special_tokens=False))
print("\n" + "=" * 80)
print("ADAPTER MODEL OUTPUT:")
print("=" * 80)
print(tokenizer.decode(outputs[0], skip_special_tokens=False))
print("\n" + "=" * 80)






BASE MODEL OUTPUT:
<bos><start_of_turn>user
What is the sum of the first 100 natural numbers?<end_of_turn>
<start_of_turn>model
Let $S$ be the sum of the first 100 natural numbers. Then we have
$$ S = 1 + 2 + 3 + \dots + 98 + 99 + 100 $$
We can use the formula for the sum of the first $n$ natural numbers, which is given by
$$ \sum_{i=1}^n i = \frac{n(n+1)}{2} $$
In our case, $n = 100$, so we have
$$ S = \frac{100(100+1)}{2} = \frac{100(101)}{2} = \frac{10100}{2} = 5050 $$
Therefore, the sum of the first 100 natural numbers is 5050.

We can also write the sum as
$$ S = \sum_{i=1}^{100} i = \frac{100(100+1)}{2} = \frac{100(101)}{2} = \frac{10100}{2} = 5050 $$
Thus, the sum of the first 100 natural numbers is 5050.

Final Answer: The final answer is $\boxed{5050}$<end_of_turn>

ADAPTER MODEL OUTPUT:
<bos><start_of_turn>user
What is the sum of the first 100 natural numbers?<end_of_turn>
<start_of_turn>model
<think>
We need to compute the sum of the first 100 natural numbers. Recall that th

In [ ]:
from datasets import load_dataset
from lib.reasoning_format import CoTFormatEvalCallback

mbpp = load_dataset("mbpp", split="test")
questions = [mbpp[i]["text"] for i in range(8)]

callback = CoTFormatEvalCallback(tokenizer, questions=questions)


In [9]:



responses, percentage = callback.evaluate_format(model)

The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [10]:
percentage

87.5

In [11]:
for response in responses:
    print(response)
    print("-" * 80)

<think>
Okay, the user wants a Python function that removes the first and last occurrences of a given character from a string. Let me think about how to approach this.

First, I need to define the function. The function should take a string as input and return a new string with the specified character removed. The function should handle cases where the character might not be present in the string.

I should consider the different scenarios: if the character is not in the string, return the original string. If the character is present, remove it from the beginning and end. Let me check if there's a way to do this efficiently.

Maybe using string slicing. For example, if the character is at the beginning, I can use slicing to remove it from the beginning. Similarly, if it's at the end, slicing from the end. But if it's in the middle, I might need to iterate through the string. Let me think about how to handle the middle case.

Alternatively, using a loop. But string slicing is more conci